In [5]:
# Run following commented section once to install if using as .ipynb
# Ignore Otherwise
# import sys
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install tensorflow_hub
# !{sys.executable} -m pip install tensorflow_datasets

import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub # These imports will not work unless you've already installed them locally. Colab has them ready.
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)


# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0


In [6]:
# Tokenize all text data

tokenizer_train = []
train_label = []
for idx, (sent, label) in enumerate(train_data):
    sent = sent.numpy()
    new_sent = [word for word in sent.lower().split()]
    tokenizer_train.append(new_sent)
    train_label.append([label.numpy()])

tokenizer_valid = []
valid_label = []
for idx, (sent, label) in enumerate(validation_data):
    sent = sent.numpy()
    new_sent = [word for word in sent.lower().split()]  
    tokenizer_valid.append(new_sent)
    valid_label.append([label.numpy()])
    
tokenizer_test = []
test_label = []
for idx, (sent, label) in enumerate(test_data):
    sent = sent.numpy()
    new_sent = [word for word in sent.lower().split()]
    tokenizer_test.append(new_sent)
    test_label.append([label.numpy()])



In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenizer_train)
word_to_ix = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(tokenizer_train)
max_len = max([len(x) for x in train_sequences])
train_padded = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=max_len)

valid_sequences = tokenizer.texts_to_sequences(tokenizer_valid)
valid_padded = pad_sequences(valid_sequences, padding='post', truncating='post', maxlen=max_len)

test_sequences = tokenizer.texts_to_sequences(tokenizer_test)
test_padded = pad_sequences(test_sequences, padding='post', truncating='post', maxlen=max_len)


In [8]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(len(word_to_ix) + 1, 64, input_length=max_len),
                                    tf.keras.layers.GlobalAveragePooling1D(),                                    
                                    tf.keras.layers.Dense(10, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

2022-04-27 20:15:45.128221: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46879488 exceeds 10% of free system memory.
2022-04-27 20:15:45.200174: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46879488 exceeds 10% of free system memory.
2022-04-27 20:15:45.223710: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46879488 exceeds 10% of free system memory.


In [10]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2470, 64)          11719872  
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 11,720,533
Trainable params: 11,720,533
Non-trainable params: 0
_________________________________________________________________


In [11]:
train_label = np.array(train_label)
valid_label = np.array(valid_label)
test_label = np.array(test_label)

model.fit(train_padded, train_label, epochs=8, validation_data=(valid_padded, valid_label))


2022-04-27 20:16:31.714052: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 148200000 exceeds 10% of free system memory.


Epoch 1/8


2022-04-27 20:16:32.521862: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46879488 exceeds 10% of free system memory.


469/469 [==============================] - 90s 190ms/step - loss: 0.6923 - accuracy: 0.5202 - val_loss: 0.6886 - val_accuracy: 0.5654
Epoch 2/8
469/469 [==============================] - 88s 188ms/step - loss: 0.6558 - accuracy: 0.6455 - val_loss: 0.6107 - val_accuracy: 0.6454
Epoch 3/8
469/469 [==============================] - 88s 188ms/step - loss: 0.5024 - accuracy: 0.8022 - val_loss: 0.4498 - val_accuracy: 0.8251
Epoch 4/8
469/469 [==============================] - 90s 191ms/step - loss: 0.3614 - accuracy: 0.8687 - val_loss: 0.3931 - val_accuracy: 0.8331
Epoch 5/8
469/469 [==============================] - 88s 187ms/step - loss: 0.2742 - accuracy: 0.9084 - val_loss: 0.3687 - val_accuracy: 0.8379
Epoch 6/8
469/469 [==============================] - 89s 189ms/step - loss: 0.2188 - accuracy: 0.9276 - val_loss: 0.3210 - val_accuracy: 0.8761
Epoch 7/8
469/469 [==============================] - 88s 188ms/step - loss: 0.1766 - accuracy: 0.9435 - val_loss: 0.3421 - val_accuracy: 0.8563
Ep

In [12]:
model.evaluate(test_padded, test_label, verbose=2)

782/782 - 4s - loss: 0.3334 - accuracy: 0.8648 - 4s/epoch - 5ms/step


[0.33343422412872314, 0.864799976348877]